# KNN Results

# Random Forest Results

# ANN Results

In [2]:
# . /home/khanhi83/.local/share/virtualenvs/Rotorcraft-Safety-BOPmdVWD/bin/activate
# To activate this project's virtualenv, run pipenv shell.
# Alternatively, run a command inside the virtualenv with pipenv run.
# import structlog
# !which python
# !conda install -c conda-forge structlog
# !pip install typing_extensions
# import structlog

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# For mutliple devices (GPUs: 4, 5, 6, 7)
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

import os
import zipfile
from typing import List, Tuple, Dict
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
print("NP:", np.__version__)
print("PD:", pd.__version__)
print("Request:", requests.__version__)
print("SKLearn:", sklearn.__version__)

plt.rcParams.update({'figure.figsize': (16.0, 12.0)})
HEADER_COLUMN = 12
LABEL_COLUMN = 'False Warning'
TEXT_COLUMN = 'Text'


Bad key "text.kerning_factor" on line 4 in
/home/khanhi83/anaconda3/envs/PY3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


NP: 1.18.1
PD: 1.0.1
Request: 2.22.0
SKLearn: 0.22.1


In [2]:
def download_file(url: str, local_dir: str = '.', local_filename: str = '') -> str:
    """
    Downloads a file from a provided url to a local directory
    :param url: URL to download the file from
    :param local_dir: Local directory to download the file to (created if it does not exist)
    :param local_filename: What to name the file when saved
     (if empty or none, assume the name of the original name of the file)
    :return: the name of the file which was saved
    """
    os.makedirs(f'{local_dir}', exist_ok=True)
    local_filename = local_filename if local_filename else url.split('/')[-1]
    if os.path.exists(f'{local_dir}/{local_filename}'):
        print("{0}/{1} already exists. Skipping download.".format(local_dir, local_filename))
    else:
        print("Downloading file from {0} to {1}/{2}.".format(url, local_dir, local_filename))
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(f'./{local_dir}/{local_filename}', 'wb') as f:
                for chunk in r.iter_content(chunk_size=128):
                    f.write(chunk)
        print("Finished saving file from {0} to {1}/{2}.".format(url, local_dir, local_filename))
    return f'{local_dir}/{local_filename}'


def unzip_file(path_to_zip_file: str, dir_to_extract_to: str) -> str:
    """
    Unzips a zip file to a provided directory
    :param path_to_file: path to zip file
    :param dir_to_extract_to: directory to extract zip file
    :return: full path to unzipped file (assuming there is only one)
    """
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(dir_to_extract_to)
        return f'{dir_to_extract_to}/{zip_ref.namelist()[0]}'


def load_data(path_to_file: str) -> pd.DataFrame:
    """
    Loads excel data from a supplied path into a Pandas dataframe
    :param path_to_file: path to excel file
    :return: Pandas dataframe containing contents of excel spreadsheet
    """
    print("Started loading the excel data from {0} into a datafram - this may take a while. You may want to grab a coffee.".format(path_to_file))
    df = pd.read_excel(path_to_file, engine='openpyxl', header=HEADER_COLUMN)
    print("Finished loading the excel data from {0} into a dataframe.".format(path_to_file))
    return df


def vectorize(df: pd.DataFrame, **kwargs) -> Tuple[np.array, List[str]]:
    print("Converting text to feature matrix")
    vectorizer = TfidfVectorizer(**kwargs)
    sparse_matrix = vectorizer.fit_transform(df[TEXT_COLUMN])
    feature_matrix = sparse_matrix.todense()
    return feature_matrix, vectorizer.get_feature_names()


def extract_and_encode_labels(df: pd.DataFrame) -> Tuple[np.array, Dict[str, int]]:
    label_mapping = dict((label, i) for i, label in enumerate(df[LABEL_COLUMN].unique()))
    labels = list(df[LABEL_COLUMN].map(label_mapping))
    return np.array(labels), label_mapping

def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements

In [3]:
if __name__ == "__main__":
    local_dir = './data'

    compute_features = not os.path.exists(f'{local_dir}/feature_data.csv')
    model_type = "mlp" #"{knn", "mlp", "rf"}  

    if compute_features:
        # download the file
        path_to_downloaded_zip_file = download_file(
            'https://www.fire.tc.faa.gov/zip/MasterModelVersion3DDeliverable.zip',
            local_dir)
        # unzip the file
        path_to_file = unzip_file(path_to_downloaded_zip_file, local_dir)

        # load the file into a Pandas dataframe
        df = load_data(path_to_file)

        # save preprocessed data to save time for future runs
        df.to_csv(f'{local_dir}/feature_data.csv')
    else:
        # don't go through the hassle of preprocessing if we already have the preprocessed data saved
        df = pd.read_csv(f'{local_dir}/feature_data.csv')

    count_of_no_text = len(df[df[TEXT_COLUMN].isnull()])
    df = df.dropna(subset=[TEXT_COLUMN])
    print("Dropped {0} records because {1} was null or NaN".format(count_of_no_text, TEXT_COLUMN))

    count_of_null_labels = len(df[df[LABEL_COLUMN].isnull()])
    df = df.dropna(subset=[LABEL_COLUMN])
    print("Dropped {0} records because {1} was null or NaN".format(count_of_null_labels, LABEL_COLUMN))

    # create a sparse feature matrix of size n x m,
    # where n = number of documents, m = number of words in vocabulary
    feature_matrix, feature_names = vectorize(df, min_df=0.001)

    labels, label_mapping = extract_and_encode_labels(df)
    num_labels = len(label_mapping)
    num_features = feature_matrix.shape[1]

    X_train, X_test, y_train, y_test = train_test_split(feature_matrix, labels, test_size=0.05, random_state=1)

    print("Training on {0} sample, validating on {1} samples".format(X_train.shape[0], X_test.shape[0]))
    print("Number of features: {0}".format(num_features))

    if model_type == "mlp":
        print("MLP training: This may take a while. You may want to grab a coffee.")
        model = MLPClassifier(hidden_layer_sizes=(num_features, 256, 128, 64, 32, num_labels),
                              #(8192, 2048, 512, 128, 32, 8, num_labels), 
                  activation='relu', solver='adam', verbose=1, learning_rate_init=0.01, batch_size=4096,
                              max_iter=25)
        model.fit(X_train, y_train)
        predict_train = model.predict(X_train)
        predict_test = model.predict(X_test)
        training_acc = confusion_matrix(predict_train, y_train)
        validation_acc = confusion_matrix(predict_test, y_test)
        print("Training accuracy with MLP: {0}".format(accuracy(training_acc)))
        print("Validation accuracy with MLP: {0}".format(accuracy(validation_acc)))
    elif model_type == "rf":
        rf = RandomForestClassifier(n_jobs=-1)
        rf.fit(X_train, y_train)
        training_acc = rf.score(X_train, y_train)
        validation_acc = rf.score(X_test, y_test)
        print("Training accuracy with Random Forest: {0}".format(training_acc))
        print("Validation accuracy with Random Forest: {0}".format(validation_acc))
    elif model_type == "knn":
        knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
        knn.fit(X_train, y_train)
        training_acc = knn.score(X_train, y_train)
        validation_acc = knn.score(X_test, y_test)
        print("Training accuracy with kNN: {0}".format(training_acc))
        print("Validation accuracy with kNN: {0}".format(validation_acc))

/home/khanhi83/anaconda3/envs/PY3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,5,9,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,87,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,117,118,119,121,122,123,124,125,126,127,128,130,131,132,145,146,148,215,224) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Dropped 3198 records because Text was null or NaN
Dropped 4438 records because False Warning was null or NaN
Converting text to feature matrix
Training on 17707 sample, validating on 932 samples
Number of features: 2285
MLP training: This may take a while. You may want to grab a coffee.
Iteration 1, loss = 1.20353836
Iteration 2, loss = 0.37653667
Iteration 3, loss = 0.28721284
Iteration 4, loss = 0.24405227
Iteration 5, loss = 0.20639902
Iteration 6, loss = 0.16921061
Iteration 7, loss = 0.14159178
Iteration 8, loss = 0.12019215
Iteration 9, loss = 0.10609428
Iteration 10, loss = 0.09564276
Iteration 11, loss = 0.08727815
Iteration 12, loss = 0.08054414
Iteration 13, loss = 0.07615574
Iteration 14, loss = 0.07225505
Iteration 15, loss = 0.06736530
Iteration 16, loss = 0.06054300
Iteration 17, loss = 0.05184826
Iteration 18, loss = 0.04625869
Iteration 19, loss = 0.03959450
Iteration 20, loss = 0.03449475
Iteration 21, loss = 0.03045931
Iteration 22, loss = 0.02841904
Iteration 23, los

/home/khanhi83/anaconda3/envs/PY3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Training accuracy with MLP: 0.9875755350991133
Validation accuracy with MLP: 0.9345493562231759
